In [1]:
from bs4 import BeautifulSoup
import mechanicalsoup
import requests
import time, os
from selenium import webdriver
import math
import pickle
import re
import numpy as np
import pandas as pd
import psycopg2 as pg
import pandas.io.sql as pd_sql

chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options

In [2]:
driver = webdriver.Chrome(chromedriver)
driver.get("https://www.tummee.com/login?")

cookies = pickle.load(open("cookies.pickle", "rb"))
for cookie in cookies:
    driver.add_cookie(cookie)

driver.get("https://www.tummee.com/sequences/teachers/daily")

In [3]:
f = open("vinyasa_urls", "rb")
vinyasa_urls = pickle.load(f)
f.close()

In [4]:
def get_poses(url): 
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    try: 
        title = re.sub(r"[\t\n0-9.]", "", soup.find("h1").text.strip())
        find_poses = soup.find_all("a", class_="english-title")
        poses = [re.sub(r"[\t\n0-9.]", "", i.text.strip()) for i in find_poses]
        class_type = "Vinyasa"
    except AttributeError:
        info = {"Title": np.nan}
    else: 
        info = {"Title": title, "Poses": poses, "Class Type": class_type}
    return info

In [5]:
len(vinyasa_urls)

20816

In [6]:
vinyasa_classes_1 = [get_poses(url) for url in vinyasa_urls[13000:15000]]

In [7]:
df = pd.DataFrame(vinyasa_classes_1)

In [8]:
vinyasa_classes_2 = [get_poses(url) for url in vinyasa_urls[15000:17000]]

In [9]:
df2 = pd.DataFrame(vinyasa_classes_2)

In [11]:
vinyasa_classes_3 = [get_poses(url) for url in vinyasa_urls[17000:]]

In [12]:
df3 = pd.DataFrame(vinyasa_classes_3)

In [13]:
vinyasa_df = pd.concat([df, df2, df3])

In [14]:
f = open("vinyasa_df", "wb")
pickle.dump(vinyasa_df, f)
f.close()